# 반드시 처음부터 끝까지 스켈레톤 코드를 살펴보고 구현하기 시작하길 바란다

## 1. 스켈레톤 코드를 [복사 및 편집] 하여 사용한다.
## 2. 아래의 [Empty Module 3개]를 직접 구현한다.


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-tp2-spam/train.csv
/kaggle/input/2021-ml-tp2-spam/test.csv
/kaggle/input/2021-ml-tp2-spam/sample_submit.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

- nltk 라이브러리에서 punkt 데이터를 다운 받음, 이 데이터는 영화 리뷰와 같은 문서 데이터로 문자의 tokeninizer를 위해서 필요하다
- nltk 라이브러리를 이용해서 불용어를 다운 받음

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df_train = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/train.csv",encoding="latin-1")
df_test = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/test.csv",encoding="latin-1")
df_train.Data

0       No I'm in the same boat. Still here at my moms...
1       (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
2          They r giving a second chance to rahul dengra.
3          O i played smash bros  &lt;#&gt;  religiously.
4       PRIVATE! Your 2003 Account Statement for 07973...
                              ...                        
4452    I came hostel. I m going to sleep. Plz call me...
4453                               Sorry, I'll call later
4454        Prabha..i'm soryda..realy..frm heart i'm sory
4455                           Nt joking seriously i told
4456                  In work now. Going have in few min.
Name: Data, Length: 4457, dtype: object

In [5]:
X_train = df_train["Data"]
y_train=df_train["Class"]
X_test=df_test["Data"]

# [Empty Module #1] 텍스트 데이터 전처리  

목표: 텍스트 데이터를 처리하기 위한 여러 과정을 거쳐, 머신을 위한 데이터를 만든다. 

```
[input]
--------------
- text: 텍스트 문장 데이터 

[output]
--------------
- text: 전처리를 완료한 문장 데이터 
    
```

In [6]:
# ------------------------------------------------
# [Empty Module #1] 텍스트 데이터 전처리
# ------------------------------------------------
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

def data_processing(text):
    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # [1] re.sub 사용해 text 속 '[^A-Za-z]' 외의 문자만을 찾아내 제거한후, pre_words 변수에 저장
    #      1) pattern은 '[^A-Za-z]', repl=' ' 로 각각 설정.
    #      2) 이모지나 숫자,점과 같은 문자외의 것들을 제거했다. (이모지는 감정 분석과 관련해서 몇가지 의미를 나타내지만 이 테스크에서는 이러한 의미도 제거함.)
    #
    # [2] pre_words의 lower 내장 함수를 이용해 대문자들은 소문자로 변경
    #      1)  대, 소문자가 구분되어 있으면 "Go"와 "go" 와 같이 동일한 단어를 머신은 다른 단어로 취급한다. 따라서 대문자를 모두 소문자로 변경.
    #
    # [3] word_tokenize 함수를 이용해 pre_word 를 토큰화하여 word를 리스트화한 후 tokenized_words변수에 저장
    #
    # [4] nltk 라이브러리로 다운 받은 stopwords의 "words" 내장 함수를 이용해 english 불용어를 찾아서 stops 변수에 저장  
    #      1) 불용어: 텍스트 분류에서 불용어는 텍스트의 중요도을 결정하는데 영향을 미치지 않는 단어임. 
    #                    (ex: the, we, a , will), 따라서 불필요한 단어가 예측 모델에 악영향을 끼칠 수 있기 때문에 제거.
    #
    # [5] [3] 에서 찾은 문자열 중 단어가 [4] 에서 찾은 불용어 속에 없을 경우, tokenized_words_remove 리스트에 append 
    #
    # [6] PorterStemmer 속 stem 내장 함수를 이용해 동일 의미를 갖는 단어를 동일한 단어로 변경하는 과정을 거친 후 다시 저장.
    #    
    # ------------------------------------------------------------
    ##############
    
    # [1] 대문자나 소문자가 아닌 문자들을 제거하여 pre_words 변수에 저장
    pre_words = re.sub('[^A-Za-z]', ' ' , text) 
    # re.sub('패턴', 교체함수, '문자열')
    # 대문자나 소문자가 아닌 문자들을 나타내는 정규표현식 : [A-Za-z] => 패턴
    # 교체함수 : ' ', 문자열 : text
    ##############
   
    # [2] pre_words내의 대문자들을 소문자로 변경
    pre_words = pre_words.lower() # .lower() 함수를 이용하여 대문자를 소문자로 변경
    ##############
    
    # [3] word_tokenize 함수를 이용해 pre_word 를 토큰화하여 word를 리스트화한 후 tokenized_words변수에 저장
    tokenized_words = word_tokenize(pre_words) # word_tokenize 함수 : space 단위와 구두점(punctuation)을 기준으로 토큰화(Tokenize)하는 함수
    ##############

    # [4] nltk 라이브러리로 다운 받은 stopwords의 "words" 내장 함수를 이용해 english 불용어를 찾아서 stops 변수에 저장 
    stops = stopwords.words('english') # .words('english') : stopwords 안의 english 불용어를 찾기
    
    ##############
    tokenized_words_remove=[] # 불용어 속에 없는 단어를 저장하는 리스트
    
    for w in tokenized_words: 
        # [5] [3] 에서 찾은 문자열 중 단어가 [4] 에서 찾은 불용어 속에 없을 경우, tokenized_words_remove 리스트에 append 
        if w not in stops: # stops 속에 tokenized_words의 단어(w)가 없을 경우
                       tokenized_words_remove.append(w) # w를 리스트에 추가
        ############## 
    
    stemmer = PorterStemmer() # PorterStemmer()를 stemmer에 저장
    # 중요도에 결정을 미치지 않는 단어인 불용어를 제외한 단어들을 저장한 리스트인 tokenized_words_remove를 사용해야함!!
    for i in range(len(tokenized_words_remove)):
        # [6] PorterStemmer 속 stem 내장 함수를 이용해 동일 의미를 갖는 단어를 동일한 단어로 변경하는 과정을 거친 후 다시 저장.
        tokenized_words_remove[i] = stemmer.stem(tokenized_words_remove[i]) # Porter stemming algorithm : 영어의 접미사(suffix)를 제거
        ##############
    
    return ( " ".join( tokenized_words_remove ))

In [7]:
X_train=[data_processing(i) for i in X_train]
X_test=[data_processing(i) for i in X_test]

# [Empty Module #2] Bag of Word 

목표: 문장 데이터를 머신을 학습하기 위한 실수화된 Feature로 변경하기로한다. 

- train 과 test 데이터 전부 type을 ('U')로 변경하여 Countvectorizer를 사용한다. 
- [설명 링크](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer)

In [8]:
# hashing vectorizer- Tfidf vectorizer가 성능측면으로 더욱 뛰어나서 Tfidf vectorizer를 사용하였습니다 => [[baseline 1]]

# ------------------------------------------------
# [Empty Module #2] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
from sklearn.feature_extraction.text import HashingVectorizer
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  CountVectorizer를 정의 
#           1) max_features를 100으로 지정 
# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
#
# [3] CountVectorizer를 이용해 X_train은 학습 및 변환을 하고, X_test는 변환을 진행. 
# ------------------------------------------------------------
###########

# # [1]
vectorizer = HashingVectorizer() # HashingVectorizer 모듈 초기화
             
# [2] X_train과 X_test를 np.array형태로 변환한 후 데이터 타입을 전부 'U'로 변경
X_train = np.asarray(X_train, dtype = 'U')
X_test = np.asarray(X_test, dtype = 'U')

# [3] HashingVectorizer로 x_train를 학습 및 변환하고, x_test를 변환
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [9]:
# count vectorizer- Tfidf vectorizer가 성능측면으로 더욱 뛰어나서 Tfidf vectorizer를 사용하였습니다 => [[baseline 3]]

# ------------------------------------------------
# [Empty Module #2] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
from sklearn.feature_extraction.text import CountVectorizer
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  CountVectorizer를 정의 
#           1) max_features를 100으로 지정 
# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
#
# [3] CountVectorizer를 이용해 X_train은 학습 및 변환을 하고, X_test는 변환을 진행. 
# ------------------------------------------------------------
###########

# [1]
vectorizer = CountVectorizer(max_features = 100) # 실수 데이터를 범주형 데이터로 바꾸기
             
# [2] X_train과 X_test를 np.array형태로 변환한 후 데이터 타입을 전부 'U'로 변경
X_train = np.asarray(X_train, dtype = 'U')
X_test = np.asarray(X_test, dtype = 'U')

# [3] CountVectorizer로 x_train를 학습 및 변환하고, x_test를 변환
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [10]:
# Tfidf Vectorizer => [[baseline 2]]

# ------------------------------------------------
# [Empty Module #2] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  CountVectorizer를 정의 
#           1) max_features를 100으로 지정 
# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
#
# [3] CountVectorizer를 이용해 X_train은 학습 및 변환을 하고, X_test는 변환을 진행. 
# ------------------------------------------------------------
###########

# [1] TfidfVectorizer를 정의
vectorizer = TfidfVectorizer(max_features = 9000) # max_feature : 용어 빈도에 의해 정렬된 상위 max_features만 고려하는 어휘를 구축
# max_feature를 높여보며 성능 향상을 시켜보고자 하였습니다
# max_feature = 1000 => 제출 성능 : 0.97937
# max_feature = 3000 => 제출 성능 : 0.98116
# max_feature = 5000 => 제출 성능 : 0.98295
# max_feature = 7000 => 제출 성능 : 0.98475
# max_feature = 9000 => 제출 성능 : 0.98475 => 선택!
             
# [2] train data와 test data의 데이터 타입을 U(=Unicode string)로 변경하고 np.array로 변환
X_train = np.asarray(X_train, dtype = 'U')
X_test = np.asarray(X_test, dtype = 'U')

# [3] train data로 학습 및 변환하고, train data로 학습한 것을 이용하여 test data를 변환
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [11]:
y_train[y_train=="ham"]=0
y_train[y_train=="spam"]=1
y_train=y_train.astype("uint8")

## [Empty Module #3] SVM: classifier
목표: SVC() 를 활용해 classification 을 진행

- fit()으로 train 에 대한 머신러닝 학습
- predict()으로 test 에 대한 정답을 추론 하여 반환

In [12]:
# ------------------------------------------------
# [Empty Module #3] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  SVC 정의 
#           1) gamma="auto" 사용 
# [2] X_train_features과 y_train으로 SVC 학습진행 후, X_test_features로 predict 진행
#
# ------------------------------------------------------------
###########
# [1]

# grid search로 최적 모델 찾기 << 파라미터를 조정하는 데 척도로 사용하였습니다
# param = {'C' : [1000, 2000, 3000, 4000, 5000]} # grid search의 파라미터 지정

## svc
svc = SVC(gamma = 'auto', kernel = 'rbf', class_weight = 'balanced', C = 10100, probability=True) 
# svc 정의 : 파라미터를 바꿔보며 제출성능을 측정해 본 결과 중 가장 좋은 파라미터로 초기화

# clf = GridSearchCV(svc, param, n_jobs = -1) # grid search 정의 => svc 파라미터를 조정하는 과정에서 사용하였습니다.

## SVC 성능 비교 ##
# param = {'C' : [0.001, 0.1, 1, 10, 100, 1000]}
# 위 파라미터로 C값을 조정해보았을 때, 
# grid search best score : 0.9807044032754048
# best_params_ : {'C': 100}
# 제출 성능 : 0.98206
# 로 C값이 클수록 성능이 좋아짐을 알 수 있었습니다.

# 따라서 param = {'C' : [1000, 2000, 3000, 4000, 5000]} 로 파라미터를 수정하고 grid search를 해보았을 때
# grid search best score : 0.9822744132908557
# best_params_ : {'C': 4000}
# 제출 성능 : 0.98295
# 을 보고 C값을 더 키워보았습니다.

# C = 7000 => 제출 성능 : 0.98295 << C = 4000이었을 때와 동일
# C = 10000 => 제출 성능 : 0.98475 << 최고 성능..!
# C = 12000 => 제출 성능 : 0.98206 << C = 10000이었을 때보다 떨어짐
# C = 11000 => 제출 성능 : 0.98206 << C = 12000이었을 때와 동일
# C = 10010 => 제출 성능 : 0.98475 << C = 10000이었을 때와 동일 
# C = 9000 => 제출 성능 : 0.98295
# C = 10100 => 제출 성능 : 0.98475
# C = 9500 => 제출 성능 : 0.98295
## 위 처럼 C값을 조정해보며 성능을 비교해본 결과 C값이 10000~10100 일 때 가장 좋은 성능을 보였고, 10100보다 큰 C값이나, 10000보다 낮은 C값을 가질 때 성능이 낮아짐을 알 수 있었습니다.

# 추가적인 성능 향상을 위해 empty module #2에서 max_feature 값을 높여보며 제출 성능 향상 추이를 보았고, max_feature가 7000과 9000일 때 같은 성능을 보임을 확인하였습니다.
# max_feature를 9000으로 유지한 채 SVC의 C파라미터를 조정해보며 성능 향상 추이를 보았습니다.
## max_feature = 9000일 때
# C = 10100 => 제출 성능 : 0.98475 ==>> best # .score : 0.9991025353376711
# C = 15000 => 제출 성능 : 0.98385
# C = 20000 => 제출 성능 : 0.98295
# C = 5000 => 제출 성능 : 0.98295
# C = 30000 => 제출 성능 : 0.98385
# C = 28000 => 제출 성능 : 0.98385
# C = 25000 => 제출 성능 : 0.98385

## random forest
rfc = RandomForestClassifier(random_state = 1, n_estimators = 50)
# rfc 
# default => 제출성능 : 0.97937
# criterion = 'entropy', n_estimators = 200 => 제출성능 : 0.97757
# n_estimators = 500 => 제출성능 : 0.97937
# n_estimators = 1000 => 제출성능 : 0.97937
# n_estimators = 50 => 제출성능 : 0.97937

## gradient boosting
boosting = GradientBoostingClassifier(random_state = 1, n_estimators = 1000)
# 제출성능 : 0.97309

## adaboost
ada = AdaBoostClassifier(random_state = 1, n_estimators = 500)
# n_estimators = 500 => 0.98026
# n_estimators = 1000 => 0.97937
# n_estimators = 300 => 0.97847

## bagging
bagging = BaggingClassifier(random_state = 1, n_estimators = 1500)
# 제출성능 : 0.97040

## voting
eclf = VotingClassifier(estimators=[('svc', svc), ('ada', ada)], voting='soft')
# rfc, svc, boosting => .score : 0.9995512676688355
# rfc, svc, ada => .score : 0.9997756338344178 , 제출성능 : 0.98475

## 성능 향상을 위해 위의 앙상블 모델을 사용해보았지만, svc 모델 하나만 쓰는 것이 가장 성능이 잘 나왔습니다.

# [2] X_train_features과 y_train으로 SVC 학습진행 후, X_test_features로 predict 진행
svc.fit(X_train_features, y_train) # train data로 svc 학습
print(svc.score(X_train_features, y_train))

# grid search로 진행했을 때
# best_model = clf.best_estimator_ # best_model에 grid search로 찾은 최적의 모델 저장
# print(clf.best_score_)
# print(clf.best_params_)

# test data에 대한 예측값 저장
y_pred = svc.predict(X_test_features) 

0.9991025353376711


# Predict to CSV

In [13]:
df_pred={"ID": range(np.array(y_pred).shape[0]),"Class":y_pred}
df_pred=pd.DataFrame(df_pred)
df_pred.to_csv("predict.csv",index=False)